This is an example of train and test pipeline for PaiNN model from schnetpack library.  
Same task could be performed with pre-defined config from repository root:
```bash
python run.py --config-name painn.yaml
```
For detailed description please refer to [README](../nablaDFT/README.md).


# Train/test cycles example using PaiNN

In [ ]:
# Based on https://github.com/atomistic-machine-learning/schnetpack/blob/master/examples/tutorials/tutorial_02_qm9.ipynb
import os

import pytorch_lightning as pl
import schnetpack as spk
import schnetpack.representation as rep
import schnetpack.task as task
import schnetpack.transform as trn
import torch
import torchmetrics
from nablaDFT import model_registry
from nablaDFT.ase_model import AtomisticTaskFixed
from nablaDFT.dataset import ASENablaDFT, dataset_registry
from nablaDFT.dataset.split import TestSplit
from nablaDFT.utils import seed_everything
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [ ]:
dataset_name = "dataset_train_tiny"  # Name of the training dataset
datapath = "database"  # Path to the selected dataset
logspath = "logs"  # Path to log files
nepochs = 200  # Number of epochs to train for
seed = 1799  # Random seed number for reproducibility
batch_size = 32  # Size of each batch for training
train_ratio = 0.9  # Part of dataset used for training
val_ratio = 0.1  # Part of dataset used for validation
n_interactions = 6  # Number of interactions to consider between atoms
n_atom_basis = 128  # Number of basis functions for atoms in the representation
n_rbf = 100  # Number of radial basis functions in the representation
cutoff = 5.0  # Cutoff distance (in Bohr) for computing interactions
devices = 1  # Number of GPU/TPU/CPU devices to use for training
transforms = [
    trn.ASENeighborList(cutoff=cutoff),
    trn.RemoveOffsets("energy", remove_mean=True, remove_atomrefs=False),
    trn.CastTo32(),
]  # data transforms used for training and validation

This example uses *tiny* train dataset split.  
All available dataset splits could be listed with:

In [ ]:
dataset_registry.list_datasets("energy")

## Downloading dataset

In [ ]:
seed_everything(seed)
workpath = logspath

if not os.path.exists(workpath):
    os.makedirs(workpath)

datamodule = ASENablaDFT(
    dataset_name="dataset_train_tiny",
    split="train",
    root="database",
    data_workdir=logspath,
    batch_size=batch_size,
    train_ratio=train_ratio,
    val_ratio=val_ratio,
    split_file=None,
    num_workers=4,
    train_transforms=transforms,
    val_transforms=transforms,
)

## Initializing training procedure and starting training

In [ ]:
pairwise_distance = spk.atomistic.PairwiseDistances()
radial_basis = spk.nn.radial.GaussianRBF(n_rbf=n_rbf, cutoff=cutoff)
cutoff_fn = spk.nn.cutoff.CosineCutoff(cutoff)
representation = rep.PaiNN(
    n_interactions=n_interactions, n_atom_basis=n_atom_basis, radial_basis=radial_basis, cutoff_fn=cutoff_fn
)
pred_energy = spk.atomistic.Atomwise(n_in=representation.n_atom_basis, output_key="energy")
pred_forces = spk.atomistic.Forces()
postprocessors = [trn.AddOffsets("energy", add_mean=True)]
nnpot = spk.model.NeuralNetworkPotential(
    representation=representation,
    input_modules=[pairwise_distance],
    output_modules=[pred_energy, pred_forces],
    postprocessors=postprocessors,
)
output_energy = spk.task.ModelOutput(
    name="energy", loss_fn=torch.nn.MSELoss(), loss_weight=1, metrics={"MAE": torchmetrics.MeanAbsoluteError()}
)
output_forces = spk.task.ModelOutput(
    name="forces", loss_fn=torch.nn.MSELoss(), loss_weight=1, metrics={"MAE": torchmetrics.MeanAbsoluteError()}
)

scheduler_args = {"factor": 0.8, "patience": 10, "min_lr": 1e-06}

task = AtomisticTaskFixed(
    model_name="PaiNN",
    model=nnpot,
    outputs=[output_energy, output_forces],
    optimizer_cls=torch.optim.AdamW,
    optimizer_args={"lr": 1e-4},
    scheduler_cls=ReduceLROnPlateau,
    scheduler_args=scheduler_args,
    scheduler_monitor="val_loss",
)

# create trainer
logger = pl.loggers.TensorBoardLogger(save_dir=workpath)
lr_monitor = LearningRateMonitor(logging_interval="step")
checkpoint_callback = ModelCheckpoint(
    save_top_k=1,
    monitor="val_loss",
    mode="min",
    dirpath=f"{workpath}/checkpoints",
    filename="Painn-{epoch:03d}_{val_loss:4f}",
)
callbacks = [lr_monitor, checkpoint_callback]

trainer = pl.Trainer(
    accelerator="gpu",
    devices=devices,
    callbacks=callbacks,
    logger=logger,
    default_root_dir=workpath,
    max_epochs=nepochs,
)

trainer.fit(task, datamodule=datamodule)

In [ ]:
ckpt_path = trainer.checkpoint_callback.best_model_path

## Initializing the testing procedure and computing the metric's result

We will use pretrained model for test with *model_registry* object.  
You could list all available pretrained model with:  

In [ ]:
model_registry.list_models()

In [ ]:
batch_size = 32
cutoff = 5.0
gpu = 0

if gpu == -1:
    device = "cpu"
else:
    device = 1

datamodule_test = ASENablaDFT(
    dataset_name="dataset_test_conformations_tiny",
    split="test",
    root="database_test",
    batch_size=batch_size,
    train_ratio=0.0,
    val_ratio=0.0,
    test_ratio=1.0,
    num_workers=4,
    split_file=None,
    test_transforms=[trn.ASENeighborList(cutoff=cutoff), trn.CastTo32()],
    splitting=TestSplit(),
)

trainer = pl.Trainer(accelerator="gpu", devices=device, inference_mode=False, logger=False)
model = model_registry.get_pretrained_model("lightning", "PaiNN_train_tiny")

trainer.test(model=model, datamodule=datamodule_test)